In [2]:
from bs4 import BeautifulSoup
import json
import jaconv

with open('html.txt', 'r') as f:
    html_data = f.read()
# 使用BeautifulSoup解析HTML
soup = BeautifulSoup(html_data, 'html.parser')

# 存储结果的列表
results = []

# 初始化当前的 location 变量
location = None

# 遍历HTML内容
for tag in soup.find_all(['h4', 'h3', 'div']):
    if tag.name == 'h4':  # 普通选区
        location = tag.text.strip()
    elif tag.name == 'h3':  # 比例区
        location = tag.text.strip()
    elif tag.name == 'div' and 'kohosya-info' in tag.get('class', []):
        # 遍历该地区内的每个候选人
        for candidate in tag.select('ol.distList > li > ol > li > dl'):
            # 提取候选人姓名、假名和政党
            name_tag = candidate.find('li', class_='kohosya-name')
            spans = name_tag.find_all('span')

            # 提取姓名和假名
            raw_name = name_tag.text.split('（')[0].strip()
            name_kana = spans[0].text.replace('）', '').replace('（', '').strip() if len(spans) > 0 else ""

            # 提取政党名称
            party = spans[1].text.strip() if len(spans) > 1 else ""

            # 提取问卷答案，处理可能缺失的情况
            setsumon = candidate.find_all('li', class_='setsumon')
            q1a = setsumon[0].text.split('：')[1].strip() if len(setsumon) > 0 else ""
            q1b = setsumon[1].text.split('：')[1].strip() if len(setsumon) > 1 else ""
            q2a = setsumon[2].text.split('：')[1].strip() if len(setsumon) > 2 else ""
            q2b = setsumon[3].text.split('：')[1].strip() if len(setsumon) > 3 else ""
            q3 = setsumon[4].text.split('：')[1].strip() if len(setsumon) > 4 else ""

            # 将该候选人的数据添加到结果列表
            results.append({
                'name': raw_name,
                'location': location,
                'name_kana': name_kana,
                'party': party,
                'q1a': q1a,
                'q1b': q1b,
                'q2a': q2a,
                'q2b': q2b,
                'q3': q3
            })

for i in range(len(results)):
    results[i]['name'] = jaconv.z2h(results[i]['name'].replace('　', ' '), kana=False, ascii=True, digit=True)
    results[i]['name_kana'] = jaconv.z2h(results[i]['name_kana'].replace('　', ' '), kana=False, ascii=True, digit=True)

# 将结果转换为JSON格式并输出
json_data = json.dumps(results, ensure_ascii=False, indent=2)
with open('data.json', 'w') as f:
    f.write(json_data)
    
print(len(results))

741


In [3]:
works = [i['q3'] for i in results]
# 将works分为多个100长度的列表，不足100的单独一个列表
works_list = [works[i:i+100] for i in range(0, len(works), 100)]

In [4]:
# with open(f'extract_by_GPT/6.txt', 'r', encoding='utf-8') as file:
#     content = file.read()
#     data = ast.literal_eval(content)

# for i in range(100):
#     print(data[i], works_list[6][i])

In [5]:
import ast
extracted = []
for i in range(8):
    with open(f'extract_by_GPT/{i}.txt', 'r', encoding='utf-8') as file:
        content = file.read()
        data = ast.literal_eval(content)
        # print(len(data))
        extracted.extend(data)
len(extracted)

741

In [6]:
import jaconv

# 删除每个子列表的元素字符串里的空格，并把全角字符统一为半角字符
cleaned_data = [[jaconv.z2h(item.replace(' ', ''), kana=False, ascii=True, digit=True) for item in sublist] for sublist in extracted]

print(cleaned_data)

[['沈黙の艦隊'], ['鬼滅の刃', 'キングダム'], ['サーキットの狼', 'ルパン三世', 'ドラゴンクエスト'], ['スラムダンク'], ['あしたのジョー', 'ドラえもん', 'アルプスの少女ハイジ'], ['スラムダンク'], ['風の谷のナウシカ', 'キングダム'], ['ゴルゴ13'], ['ゴールデンカムイ'], ['銀の匙'], ['ドラゴンクエスト', 'ファイナルファンタジー'], ['アンパンマン'], [], ['ガラスの仮面'], ['ゴールデンカムイ', 'コウノドリ'], ['ゴルゴ13'], ['ゴールデンカムイ', 'キングダム'], [], ['あしたのジョー', '手塚治虫作品'], ['魁!!男塾'], [], ['寄生獣', 'ルナティック雑技団', '島耕作シリーズ'], ['風の谷のナウシカ'], ['戦国時代史', 'まんが日本昔ばなし'], ['ドラゴンボール', 'スラムダンク'], ['宇宙兄弟', 'フラジャイル', '推しの子'], ['スラムダンク', '宮崎駿作品', '新海誠作品'], ['火の鳥', 'メジャー'], ['浮浪雲'], ['キングダム'], ['ハイキュー', 'バクテン', 'ゴールデンスランパー'], [], ['スパイファミリー'], ['ワンピース', '松本大洋作品'], ['スタジオジブリ作品'], ['ハイキュー!!'], ['BanGDream!', 'ウマ娘', '魔法少女まどか☆マギカ'], ['ゴルゴ13', '鬼滅の刃'], ['美味しんぼ', 'ドラゴンクエスト'], ['柔道部物語', 'ゴリラーマン', 'ゴースト・オブ・ツシマ'], ['釣りバカ日誌'], ['少女革命ウテナ', '未来警察ウラシマン', 'うる星やつら'], [], ['さよなら銀河鉄道999', 'AKIRA', '機動戦士ガンダム'], ['かろりのつやごと', 'ミステリと言う勿れ', '花より男子'], ['釣りキチ三平', '美味しんぼ', '宇宙戦艦ヤマト'], ['ブラックジャック', '包丁人味平', 'ムーミン'], [], ['転スラ', 'HUNTER×HUNTER', 'BLEACH'], ['ドラゴンボール', 'HUNTER×HUNTER',

In [7]:
for i, j in enumerate(cleaned_data):
    print(i, j)

0 ['沈黙の艦隊']
1 ['鬼滅の刃', 'キングダム']
2 ['サーキットの狼', 'ルパン三世', 'ドラゴンクエスト']
3 ['スラムダンク']
4 ['あしたのジョー', 'ドラえもん', 'アルプスの少女ハイジ']
5 ['スラムダンク']
6 ['風の谷のナウシカ', 'キングダム']
7 ['ゴルゴ13']
8 ['ゴールデンカムイ']
9 ['銀の匙']
10 ['ドラゴンクエスト', 'ファイナルファンタジー']
11 ['アンパンマン']
12 []
13 ['ガラスの仮面']
14 ['ゴールデンカムイ', 'コウノドリ']
15 ['ゴルゴ13']
16 ['ゴールデンカムイ', 'キングダム']
17 []
18 ['あしたのジョー', '手塚治虫作品']
19 ['魁!!男塾']
20 []
21 ['寄生獣', 'ルナティック雑技団', '島耕作シリーズ']
22 ['風の谷のナウシカ']
23 ['戦国時代史', 'まんが日本昔ばなし']
24 ['ドラゴンボール', 'スラムダンク']
25 ['宇宙兄弟', 'フラジャイル', '推しの子']
26 ['スラムダンク', '宮崎駿作品', '新海誠作品']
27 ['火の鳥', 'メジャー']
28 ['浮浪雲']
29 ['キングダム']
30 ['ハイキュー', 'バクテン', 'ゴールデンスランパー']
31 []
32 ['スパイファミリー']
33 ['ワンピース', '松本大洋作品']
34 ['スタジオジブリ作品']
35 ['ハイキュー!!']
36 ['BanGDream!', 'ウマ娘', '魔法少女まどか☆マギカ']
37 ['ゴルゴ13', '鬼滅の刃']
38 ['美味しんぼ', 'ドラゴンクエスト']
39 ['柔道部物語', 'ゴリラーマン', 'ゴースト・オブ・ツシマ']
40 ['釣りバカ日誌']
41 ['少女革命ウテナ', '未来警察ウラシマン', 'うる星やつら']
42 []
43 ['さよなら銀河鉄道999', 'AKIRA', '機動戦士ガンダム']
44 ['かろりのつやごと', 'ミステリと言う勿れ', '花より男子']
45 ['釣りキチ三平', '美味しんぼ', '宇宙戦艦ヤマト']
46 ['ブラックジャック', '包

In [8]:
rplc_dict = {'こち亀': 'こちら葛飾区亀有公園前派出所',
             'おーい竜馬': 'お~い!竜馬',
             'おーい龍馬': 'お~い!竜馬',
             'きのう何食べた': 'きのう何食べた?',
             'Re': 'Re:ゼロから始める異世界生活',
             'リゼロ': 'Re:ゼロから始める異世界生活',
             'Monster': 'MONSTER',
             'HUNTER✖HUNTER': 'HUNTER×HUNTER',
             'OnePiece': 'ONE PIECE',
             'ONEPIECE': 'ONE PIECE',
             'ワンピース': 'ONE PIECE',
             'ナルト': 'NARUTO',
             'YAWARA': 'YAWARA!',
             'Dr.ストーン': 'Dr.STONE',
             'ソードアートオンライン': 'SAO',
             'スーパーマリオブラザース': 'スーパーマリオブラザーズ',
             'スポーツ系漫画': 'スポーツ系作品',
             'ジブリアニメ': 'ジブリ作品',
             'スタジオジブリ作品': 'ジブリ作品',
             'シティハンター': 'シティーハンター',
             '桃鉄': '桃太郎電鉄',
             'もも鉄': '桃太郎電鉄',
             '攻殻機動隊': '攻殻機動隊シリーズ',
             '課長島耕作': '島耕作シリーズ',
             '島耕作': '島耕作シリーズ',
             '聖おにいさん': '聖☆おにいさん',
             '火の鳥シリーズ': '火の鳥',
             '歴史もの': '歴史系作品',
             'ゲゲゲのキタロウ': 'ゲゲゲの鬼太郎',
             'ウマ娘プリティダービー': 'ウマ娘',
             'ウマ娘プリティーダービー': 'ウマ娘',
             'エースをねらえ': 'エースをねらえ!',
             'ガンダム': '機動戦士ガンダム',
             'ガンダムシリーズ': '機動戦士ガンダム'
             'ガンダムSEED': '機動戦士ガンダムSEED',
             'Zガンダム': '機動戦士Zガンダム',
             'ゴースト・オブ・ツシマ': 'GHOST OF TSUSHIMA',
             'ghostoftsushima': 'GHOST OF TSUSHIMA',
             'theriseofronin': 'THE RISE OF RONIN',
             'ゴールデンスランパー': 'ゴールデンスランバー',
             'ザファブル': 'ザ・ファブル',
             'ファブル': 'ザ・ファブル',
             'スーパーマリオ・ブラザーズ3': 'スーパーマリオブラザーズ3',
             'ディズニーアニメ': 'ディズニー作品',
             'ドラクエ': 'ドラゴンクエストシリーズ',
             'ドラゴンクエスト': 'ドラゴンクエストシリーズ',
             'ハイキュー': 'ハイキュー!!',
             '排球': 'ハイキュー!!',
             'ルパン': 'ルパン三世シリーズ',
             'ルパン3世': 'ルパン三世シリーズ',
             'ルパン三世': 'ルパン三世シリーズ',
             'FF': 'ファイナルファンタジーシリーズ',
             'ファイナルファンタジー': 'ファイナルファンタジーシリーズ',
             'FF7': 'ファイナルファンタジーVII',
             'ブラックジャック': 'ブラック・ジャック',
             'マインスイーパ': 'マインスイーパー',
             'スーパーマリオ': 'スーパーマリオシリーズ',
             'マリオ': 'スーパーマリオシリーズ',
             'メタルギアソリッド': 'メタルギアソリッドシリーズ',
             '一・二の三四郎': '1・2の三四郎',
             '3丁目の夕日': '三丁目の夕日',
             '三月のライオン': '3月のライオン',
             '今敏監督作品': '今敏作品',
             '君の名は': '君の名は。',
             '呪術回戦': '呪術廻戦',
             '宮崎駿シリーズ': '宮崎駿作品',
             '家裁の人': '家栽の人',
             'エヴァンゲリオン': '新世紀エヴァンゲリオン',
             '藤子・F・不二雄作品': '藤子不二雄作品',
             '転スラ': '転生したらスライムだった件',
             '釣りキチさんぺい': '釣りキチ三平',
             '龍(ろん)': '龍-RON-',
             'スラムダンク': 'SLAM DUNK',
             'SLAMDUNK': 'SLAM DUNK',
             'ハイジ': 'アルプスの少女ハイジ',
             'スパイファミリー': 'SPY×FAMILY'
             }

# 按照替换词典，替换每个子列表的元素字符串
new_cleaned_data = [[rplc_dict.get(item, item) for item in sublist] for sublist in cleaned_data]

In [9]:
# add new_cleaned_data to the json(results)
for i in range(len(results)):
    results[i]['q3_format'] = new_cleaned_data[i]

with open('data_format.json', 'w') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

In [10]:
from collections import Counter

import matplotlib.pyplot as plt

# 统计每个作品出现的次数
flat_list = [item for sublist in new_cleaned_data for item in sublist]
counter = Counter(flat_list)

# 将统计结果转换为两个列表
labels, values = zip(*counter.items())

# 按照出现次数降序排序
indSort = sorted(range(len(values)), key=lambda i: values[i], reverse=True)
labels = [labels[i] for i in indSort]
values = [values[i] for i in indSort]

# #设置中文字体
# plt.rcParams['font.sans-serif'] = ['Noto Sans CJK JP']

# # 绘制统计图,忽略出现次数为1的作品
# values = [i for i in values if i > 1]
# labels = labels[:len(values)]
# plt.figure(figsize=(10, 30))  # 加长图片高度
# bars = plt.barh(labels, values)
# plt.xlabel('出现次数')
# plt.ylabel('作品标题')
# plt.title('作品出现次数统计')

# # 在柱状图旁写上具体出现次数
# for bar in bars:
#     plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2, f'{bar.get_width()}', va='center')

# plt.savefig('works.pdf')